# Aéroports du monde


En prenant pour prétexte un fichier de données recensant un très grands nombre d'aéroports, nous allons travailler  pour : 
- importer un fichier csv
- sélectionner des données
- trier des données
- fusionner des données

# 1) Pour commencer \#1: fichier que nous allons exploiter

In [ ]:
import csv

def charger_fichier( nom_fic ):
    """
    Permet de lire un fichier CSV en utilisant la ligne d'entêtes
    Retourne une liste de dictionnaires.
    """
    table = []
    fichier_csv = open( nom_fic , "r", newline ="", encoding ="utf-8" )
    lecteur_fichier = csv.DictReader( fichier_csv , delimiter =",")    
    for enreg in lecteur_fichier :
        table.append (dict ( enreg )) # enreg est de type OrderedDict on le remet endict
    fichier_csv.close()    
    
    return table

<div class = "alert alert-info">  


**Question**  


Ouvrir le fichier `.\airports\airport.csv` dans Jupyter.
    
- Noter le séparateur utilisé.  
    
    
- Modifier alors la fonction ci-dessus pour qu'elle puisse importer correctement ce fichier (il y a deux modifications à effectuer aux lignes 9 et 10).  

In [ ]:
table_aeroports = charger_fichier('./airports/airports.csv')
table_aeroports

# 2) Pour commencer \#2 : selectionner des données, afficher

<div class = "alert alert-info">  


**Question 2.6**  
On se perfectionne en création de liste par compréhension !!
Dans un premier temps à partir de la table `table_aeroports`, créer la **table** `table_large_airports` de tous les aéroports de type `large_airport`, en utilisant une ligne pour la boucle for, et une ligne pour le if !
    
   

In [ ]:
table_large_airports = [ aero for aero in table_aeroports if aero['Type'] == 'large_airport' ]
table_large_airports

# 4) Enrichir une carte avec des marqueurs

Nous allons utiliser le module `folium` qui permet de générer des cartes géographiques interactives (zoomables) avec des marqueurs légendés (cliquables). Lorsqu'on cherche à générer une carte avec ce module le processus de conception est le suivant, qui se déroule en trois étapes :

- Créer une "carte" (de type `folium.Map`)
- Ajouter des "marqueurs" sur la "carte" (de type `folium.Marker` ou `folium.CircleMarker`)
    - En spécifiant les coordonnées géographiques (latitude et longitude)
    - En spécifiant si besoin l'objet `folium.Popup` qui doit apparaître lors d'un clic sur le marqueur
    - En spécifiant d'autres paramètres tels que la couleur du marqueur, la taille du marqueur etc.
- Obtenir la carte :
    - dans Jupyter simplement demander sa représentation permet de l'afficher
    - on peut aussi la sauvegarder sous forme d'une page HTML pour être insérée dans une page web

La fonction `ajouter_marqueur` ci-dessous permet de gérer l'étape 2 (pour un seul marqueur) et prend en paramètres :
- une latitude (de type `float`)
- une longitude (de type `float`)
- une légende sous forme de dictionnaire (clés et valeurs de type `str`)

Elle permet d'ajouter un marqueur ayant la légende donnée, au point de coordonnées données, sur une folium.Map.

<div class = "alert alert-info">  


**Question 4.1**  

Exécuter la cellule de code ci-dessous.

In [ ]:
import folium




def ajouter_marqueur(carte, latitude, longitude, nom, pays, couleur):
    '''
    carte : de type folium.Map
    latitude et longitude : de type float
    dictionnaire : de type dict avec clées et valeurs de type str
    couleur : au format '#RRGGBB' où RR, GG, BB sont des entiers entre 0 et 255 en hexadécimal
              représentant les composant Rouge, Verte et Bleue de la couleur
    '''
    radius = 4
    folium.CircleMarker(
        location = [latitude, longitude],
        radius = radius,
        popup = nom  +"  "+ pays,
        color = couleur,
        fill = True
    ).add_to(carte)
    


#étape 1 : création de la carte  
ma_carte = folium.Map(location=(45, 0), zoom_start=2)

#étape 2 : ajout de quatre marqueurs
ajouter_marqueur(ma_carte,  47.8,   1.6,  'VILLORCEAU','FR', "#FF0000")
ajouter_marqueur(ma_carte,    47,    30, 'AAA','RU' ,"#880000")
ajouter_marqueur(ma_carte,    20,   -30,  'Atlantide','',   "#00FFFF")
ajouter_marqueur(ma_carte, -13.2, -72.5,  'Cité d\'Or','PE',   "#88BB88")

#étape 3 : affichage de la carte
ma_carte

#étape 3 : variante en sauvegardant la carte au format HTML (sauvegarde dans le dossier courant)
#ma_carte.save('.\carte_de_test.html')




<div class = "alert alert-info">  


**Question**  

Modifier le code ci-dessus pour :
- que les marqueurs soient en jaune (Rouge + Vert en synthèse additive),
- que les cercles tracés soient deux fois plus petit (diviser leur rayon par deux),


<div class = "alert alert-danger">  


# Et nos aéroports ?
    
Nus avons vu comment accéder au fichier des aéroports et comment enrichir une carte avec des marqueurs. Nous allons désormais chercher à créer une carte avec les aéroports de la façon suivante :
    
- La couleur du marqueur dépendra de l'altitude de l'aéroport :
    - entre 0 et 100 pieds : bleu
    - entre 100 et 2000 pieds (ente 40 et 500 m) : vert 
    - entre 2000 et 3000 pieds (500 et 1000m) : jaune
    - entre 3000 et 5000 pieds (1000m et 1500m) : rouge
    - au delà de 5000 pieds (1500 m d'altitude) : noir (équivalent : entre 5000 et 99999 pieds d'altitude)
    
    
     
    
Pour cela nous allons pour chacune des cinq plages d'altitude :
- Créer une nouvelle table ne comportant que les aéroports ayant une altitude dans cette plage,
- Pour chacun des aéroports de cette nouvelle table :
    - Utiliser la fonction `ajouter_marqueur` pour placer l'aéroport correspondant sur la carte

Bien entendu nous allons travailler avec des fonctions.
    

**Remarque :** Il aurait été possible de procéder autrement et d'optimiser un peu. Mais en travaillant ainsi on exploite davantage les capacités attendues du programme de NSI.

# 4) Quelques fonctions à créer

<div class = "alert alert-info">  


**Question**  

Compléter la fonction `extraire_aeroports_dans_plage` qui prend en paramètres :
- une table d'aéroprts `table` comportant un champ `Altitude` exprimée en pieds,
- une altitude minimale `alt_min` exprimée en mètres,
- une altitude maximale `alt_max` exprimée en mètres,
    
et renvoie une table comportant tous les aéroports ayant une altitude comprise entre `alt_min` (compris) et `alt_max` (non compris)
    
**Remarque :** en procédant avec une création de liste par comprhénsion cela peut se faire en une seule instruction (plus l'instruction `return ...`)
**Remarque :** Ne pas oublier le types des données !! ici le nombres sont des chaines de caratéres !!! donc à convertir en float !
    

**Remarque :** 1 pied = 0,3048 mètre
   

In [ ]:
def extraire_aeroports_dans_plage(table, alt_min, alt_max):
    #à compléter pour répondre
    return table_extraite

In [ ]:
extraire_aeroports_dans_plage(table_aeroports, 100, 2000)

# 5) Une première carte

<div class = "alert alert-info">  


**Question**  

On dispose désormais de deux fonctions :
- `ajouter_marqueur`
- `extraire_aeroports_dans_plage`



1) En utilisant ces deux fonctions, compléter la cellule de code ci-dessous pour obtenir une carte avec les aéroports en bleu ayant une altitude entre  0 et 100 pieds (0 et 40 mètres).
    
    
2) Compléter le code pour placer sur la carte les 4 autres catégories d'aéroports   
    - entre 0 et 100 pieds : bleu
    - entre 100 et 2000 pieds (ente 40 et 500 m) : vert 
    - entre 2000 et 3000 pieds (500 et 1000m) : jaune
    - entre 3000 et 5000 pieds (1000m et 1500m) : rouge
    - au delà de 5000 pieds (1500 m d'altitude) : noir
On pourra :
- soit créer une fonction qui reprend le code du 1) et qu'on appellera cinq fois,
- soit faire 4 Copier-Coller (solution moins élégante)

In [ ]:
table_aeroports = charger_fichier('.\\airports\\airports.csv')

#étape 1 : création de la carte  
ma_carte = folium.Map(location=(45, 0), zoom_start=2)

#étape 2 : ajout des marqueurs


    
#à compléter (pas mal de ligne de code)
    

#étape 3 : affichage de la carte
ma_carte

# 6) Une amélioration qui nécessite une fusion de tables

On souhaite afficher une variante de cette carte pour laquelle le code du nom du pays a été remplacé par son nom de pays selon la norme ISO-3166. Au lieu de FR, s'affichera France, CN : China.... C'est plus facile ! C'était l'idée d'Ange !! Non ?

Pour ce dernier point, il convient de savoir que notre table aéroports respectait les noms de pays de la nomre ISO-3166 et que l'on dispose d'une table adéquate permettant d'assurer la correspondance :

In [ ]:
import csv

def charger_fichier( nom_fic ):
    """
    Permet de lire un fichier CSV en utilisant la ligne d'entêtes
    Retourne une liste de dictionnaires.
    """
    table = []
    fichier_csv = open( nom_fic , "r", newline ="", encoding ="utf-8" )
    lecteur_fichier = csv.DictReader( fichier_csv , delimiter =",")    
    for enreg in lecteur_fichier :
        table.append (dict ( enreg )) # enreg est de type OrderedDict on le remet endict
    fichier_csv.close() 
    return table

In [ ]:
table_pays = charger_fichier('./airports/countries.csv')
table_aeroports = charger_fichier('./airports/airports.csv')

Pour l'Andorre, le code alpha-2 est 'AD' (il existe aussi un code alpha-3 sur 3 lettres):

In [ ]:
table_pays[0]

Nous allons fusionner la table des aéroports `table_aeroports` avec la table des pays `table_pays` afin d'obtenir une table fusionnée comportant tous les champs de la table `table_aeroports` plus le champ `Country_Name`

<div class = "alert alert-info">  
    

**Question**
    
    
Compléter le code de la fonction `fusionner` ci-dessous :
- prenant en paramètre les tables `table_gauche` et `table_droite` 
    
    
- renvoyant une table `table_fusionnee` telle que :
    - chaque dictionnaire de la table fusionnee comporte tous les champs de la table gauche `table_gauche` et le champ `name` de la `table_droite` renommé en champ `Country_Name`,
    
    - avec le champ `Country_Code` utilisé comme champ de fusion pour `table_gauche` et `code` comme champ de fusion pour `table_droite` 
    
    
(`table_gauche` sera la table des aéroports, `table_droite` sera la table des pays).
    
Voici un exemple d'enregistrement (de ligne) de la table fusionnée : on remarque le champ `Country_Name` en plus.

```
{'Airport_ID': '2596',
 'Name': 'Sørkjosen Airport',
 'Type': 'medium_airport',
 'City': 'Sørkjosen',
 'Country_Code': 'NO',
 'IATA': 'SOJ',
 'Latitude': '69.786796569824',
 'Longitude': '20.959400177002',
 'Altitude': '16',
 'Country_Name': 'Norway'}    
```

In [ ]:
import copy

def fusionner(table_gauche, table_droite):
    table_fusion = []
    for gauche in table_gauche:
        for droite in table_droite:
            #à compléter      
    return table_fusion

In [ ]:
table_fusionnee = fusionner (table_aeroports, table_pays)
table_fusionnee[1500]

<div class = "alert alert-info">  
    

**Question**
    
    
Modifier le code ci-dessous (à un seul endroit) pour qu'il affiche les aéroports avec le nom des pays au lieu des codes des pays comme auparavant.

In [ ]:
def extraire_aeroports_dans_plage(table, alt_min, alt_max):
    table_extraite = [aeroport for aeroport in table if
                     alt_min <=  float(aeroport['Altitude']) and
                      float(aeroport['Altitude']) < alt_max]
    return table_extraite


#étape 0 : creer la table des aeroports fusionnee
table_pays = charger_fichier('./airports/countries.csv')
table_aeroports = charger_fichier('./airports/airports.csv')
table_aeroports = fusionner (table_aeroports, table_pays)

#étape 1 : création de la carte  
ma_carte = folium.Map(location=(45, 0), zoom_start=2)

#étape 2 : ajout des marqueurs

def ajouter_plage_sur_carte(carte, table_des_aeroports, alt_min, alt_max, couleur):
    plage = extraire_aeroports_dans_plage(table_des_aeroports, alt_min, alt_max)
    for aero in plage :
        
        ajouter_marqueur(carte,  aero['Latitude'],aero['Longitude'] ,aero['Name'] ,aero['Country_Code'], couleur)   

ajouter_plage_sur_carte(ma_carte, table_aeroports,    0,    100, '#0000FF')
ajouter_plage_sur_carte(ma_carte, table_aeroports,   100,   2000, '#00FF00')
ajouter_plage_sur_carte(ma_carte, table_aeroports,  2000,  3000, '#FFFF00') 
ajouter_plage_sur_carte(ma_carte, table_aeroports, 3000,  5000, '#FF0000') 
ajouter_plage_sur_carte(ma_carte, table_aeroports, 5000, 99999, '#000000') 
    

#étape 3 : affichage de la carte
ma_carte